In [1]:
import os
import pandas as pd
import json
import folium
from numpy import interp
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster

## Datasets and topojson files

In [2]:
amstat_path = 'datasets/tasso.csv'
europe_geo_path = r'topojson/europe.topojson.json'
cantons_geo_path = r'topojson/ch-cantons.topojson.json'
geolocator = Nominatim()

## Reading csv file

In [3]:
df = pd.read_csv(amstat_path, encoding='utf_16')

In [4]:
europe_json_data = json.load(open(europe_geo_path))
cantons_json_data = json.load(open(cantons_geo_path))

## Cleaning up dataframe

In [5]:
df = df.filter(regex=('Totale' + ".*|Cantone"))
df = df.apply(lambda x: x.str.replace("'",''))
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df.columns.values[0] = 'Cantone'
df = df.set_index('Cantone')
del df['Coefficiente di variazione']
del df['Coefficiente di variazione dei giovani']
cols=[i for i in df.columns if i not in 'Cantone']
for col in cols:
    df[col]=pd.to_numeric(df[col])
df = df.drop('Totale')

## Mapping each Canton to its Sign

In [6]:
cantons = {
        'Zurigo': 'ZH',
        'Berna': 'BE',
        'Lucerna': 'LU',
        'Uri': 'UR',
        'Svitto': 'SZ',
        'Obwaldo': 'OW',
        'Nidwaldo': 'NW',
        'Glarona': 'GL',
        'Zugo': 'ZG',
        'Friburgo': 'FR',
        'Soletta': 'SO',
        'Basilea-Città': 'BS',
        'Basilea-Campagna': 'BL',
        'Sciaffusa': 'SH',
        'Appenzello Esterno': 'AR',
        'Appenzello Interno': 'AI',
        'San Gallo': 'SG',
        'Grigioni': 'GR',
        'Argovia': 'AG',
        'Turgovia': 'TG',
        'Ticino': 'TI',
        'Vaud': 'VD',
        'Vallese': 'VS',
        'Neuchâtel': 'NE',
        'Ginevra': 'GE',
        'Giura': 'JU'
           }
for index,row in df.iterrows():
    df.loc[index, 'Sign'] = cantons[index]

## Creating Folium Map

In [7]:
m = folium.Map([46.8,8], tiles='cartodbpositron', zoom_start=8)

## Adding cloropleth map

In [8]:
m.choropleth(geo_data=cantons_json_data,
             data=df,
             columns=['Sign', 'Tasso di disoccupazione dei giovani'],
             key_on='feature.id',
             fill_color='OrRd', 
             fill_opacity=0.5, 
             line_opacity=0.2,
             highlight=True,
             legend_name='Unemployement rate', topojson='objects.cantons')

## Creating HTML Popup

In [9]:
def getHtml(location_name, rate_value):
    return """
    <h3> """ + location_name + """</h3><br>
    <h4>
        Unemployement rate: <em>"""+ rate_value +""" </em>
    </h4>
    """

## Adding marker with custom popup for each canton

In [10]:
marker_cluster = MarkerCluster().add_to(m)

for index,row in df.iterrows():
    place = df.at[index,'Sign']
    if (place == 'AI'):
        place = 'Appenzell Innerrhoden'
    elif (place == 'SH'):
        place = 'Schaffhausen'
    location = geolocator.geocode(place + ', Schweiz/Suisse/Svizzera/Svizra')
    df.loc[index, 'Latitude'] = location.latitude
    df.loc[index, 'Longitude'] = location.longitude    
    html = getHtml(location.address.split(',', 1)[0], str(df.at[index,'Tasso di disoccupazione dei giovani']))
    folium.CircleMarker(
        location=[location.latitude, location.longitude],
        radius=df.at[index,'Tasso di disoccupazione dei giovani']*5,
        fill=True,
        popup=folium.Popup(html)
    ).add_to(marker_cluster)

In [11]:
m

In [12]:
m.save(os.path.join('results', 'ex2.html'))

In [13]:
df.sort_values('Tasso di disoccupazione dei giovani', ascending=False)

,Tasso di disoccupazione,Tasso di disoccupazione dei giovani,Disoccupati registrati,Entrate alla disoccupazione,Uscite della disoccupazione,Disoccupati dei giovani,Entrate alla disoccupazione giovanile,Giovani disoccupati uscite,Disoccupati di lunga durata,Persone in cerca dimpiego,Persone in cerca dimpiego non disoccupate,Persone in cerca dimpiego non disoccupate - POT,Persone in cerca dimpiego non disoccupate - altre,Persone in cerca dimpiego non disoccupate - riqualificazione/perfezionamento,Persone in cerca dimpiego non disoccupate - Guadagno intermedio,Persone in cerca dimpiego entrate,Persone in cerca dimpiego uscite,Sign,Latitude,Longitude
Cantone,,,,,,,,,,,,,,,,,,,,
Neuchâtel,5.1,6.1,4738,891,1017,726,213,229,1210,6350,1612,278,272,32,1030,642,764,NE,47.009925,6.824381
Ticino,3.1,5.2,5202,1658,1611,803,381,324,797,8675,3473,679,850,48,1896,1340,1170,TI,46.335651,8.753706
Giura,4.4,5.1,1619,292,385,266,75,114,446,2375,756,9,388,8,351,243,301,JU,47.356670,7.159889
Ginevra,5.2,4.6,12234,2260,2158,914,346,227,2326,15497,3263,601,448,191,2023,1835,1945,GE,46.223030,6.146693
Basilea-Città,3.5,4.4,3455,616,791,408,114,144,606,5168,1713,124,426,27,1136,616,824,BS,47.557910,7.592773
Vaud,4.3,4.3,17155,4195,4535,1904,702,835,3015,24649,7494,1388,1131,16,4959,3186,3391,VD,46.635696,6.532072
Sciaffusa,3.0,3.8,1286,221,240,226,52,44,268,2328,1042,88,432,34,488,219,290,SH,47.696049,8.634513
Zurigo,3.3,3.7,27225,5067,5340,3420,963,996,4753,34156,6931,295,2563,89,3984,5071,5438,ZH,47.413302,8.656394
Argovia,2.9,3.3,10684,2286,2478,1555,517,585,1698,15145,4461,389,1368,26,2678,2074,2261,AG,47.412396,8.194832
